In [51]:
import pandas as pd
import plotly 
import numpy as np
import plotly.figure_factory as ff
import scipy
import plotly.graph_objects as go
import dash
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.express as px

#pip install scipy
#!pip install Dash

In [52]:
# Import data

data = pd.read_csv('chocolate.csv')
continent = pd.read_csv('countryContinent.csv',encoding = "ISO-8859-1")
imp_exp=pd.read_csv('UNdata_Export_20220301_151116452.csv')
coord=pd.read_csv('country_points.csv',encoding = "ISO-8859-1")

In [53]:
coord.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [54]:
continent.head()

,country,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0


In [55]:
imp_exp.head()

,Table Code,Country or Area Code,Country or Area,Year,Comm. Code,Commodity,Flow Code,Flow,Trade (USD),Weight (kg),Quantity Code,Quantity Name,Quantity
0,19,4,Afghanistan,2018,180500,"Cocoa powder, unsweetened",1,Import,7.247329e+04,8.070000e+04,8,Weight in kilograms,8.070000e+04
1,19,4,Afghanistan,2018,180690,Chocolate/cocoa food preparations nes,1,Import,3.281740e+07,5.662823e+06,8,Weight in kilograms,5.662823e+06
2,19,4,Afghanistan,2018,180690,Chocolate/cocoa food preparations nes,2,Export,4.327404e+06,0.000000e+00,8,Weight in kilograms,6.239175e+06
3,19,4,Afghanistan,2018,180690,Chocolate/cocoa food preparations nes,3,Re-Export,7.452492e+04,0.000000e+00,8,Weight in kilograms,0.000000e+00
4,19,4,Afghanistan,2016,180690,Chocolate/cocoa food preparations nes,1,Import,1.568728e+08,2.677301e+07,8,Weight in kilograms,2.677301e+07


In [56]:
imp_exp=imp_exp[imp_exp['Commodity']=='Cocoa beans, whole or broken, raw or roasted'][['Country or Area','Year','Commodity','Flow','Quantity','Trade (USD)']]
imp_exp.head()

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD)
60,Albania,2012,"Cocoa beans, whole or broken, raw or roasted",Import,130.0,874.0
71,Albania,2011,"Cocoa beans, whole or broken, raw or roasted",Import,4524.0,19877.0
83,Albania,2010,"Cocoa beans, whole or broken, raw or roasted",Import,45.0,295.0
94,Albania,2009,"Cocoa beans, whole or broken, raw or roasted",Import,262.0,1919.0
149,Albania,2004,"Cocoa beans, whole or broken, raw or roasted",Import,1606.0,2729.0


In [57]:
# Prepare the data for the merge
data["company_location"] = data["company_location"].str.title()
data["country_of_bean_origin"] = data["country_of_bean_origin"].str.title()
data["company_location"].replace({'U.S.A': 'United States of America','U.K.':'United Kingdom of Great Britain and Northern Ireland','Dominican republic':'Dominican Republic','El salvador':'El Salvador','Vietnam':'Viet Nam','Venezuela':'Venezuela (Bolivarian Republic of)','South Korea':'Korea (Republic of)','New Zealand':'New Zealand','Russia':'Russian Federation','Taiwan':'Taiwan, Province of China','Sao Tome':'Sao Tome and Principe','Sao Tome & Principe':'Sao Tome and Principe','St. Lucia':'Saint Lucia','U.A.E.':'United Arab Emirates','St.Vincent-Grenadines':'Saint Vincent and the Grenadines','Bolivia':'Bolivia (Plurinational State of)'}, inplace=True)
data["country_of_bean_origin"].replace({'U.S.A': 'United States of America','U.K.':'United Kingdom of Great Britain and Northern Ireland','Dominican republic':'Dominican Republic','El salvador':'El Salvador','Vietnam':'Viet Nam','Venezuela':'Venezuela (Bolivarian Republic of)','South Korea':'Korea (Republic of)','New Zealand':'New Zealand','Russia':'Russian Federation','Taiwan':'Taiwan, Province of China','Sao Tome':'Sao Tome and Principe','Sao Tome & Principe':'Sao Tome and Principe','St. Lucia':'Saint Lucia','U.A.E.':'United Arab Emirates','St.Vincent-Grenadines':'Saint Vincent and the Grenadines','Bolivia':'Bolivia (Plurinational State of)','Burma':'Myanmar','Tanzania':'Tanzania, United Republic of','Trinidad':'Trinidad and Tobago','Dr Congo':'Congo (Democratic Republic of the)'}, inplace=True)

In [58]:
# removing Unnamed:0
data=data.iloc[:,1:]

In [59]:
teste=data.merge(continent[['country','continent','sub_region','code_2']].rename(columns={'continent':'company_continent','sub_region':'company_region','code_2':'company_code_2'}), left_on='company_location', right_on='country', how='left')

In [60]:
teste[teste['company_continent'].isna()]['company_location'].value_counts

<bound method IndexOpsMixin.value_counts of 193        Wales
486     Scotland
487     Scotland
488     Scotland
489     Scotland
490     Scotland
491     Scotland
1083    Scotland
1084    Scotland
1513    Scotland
1514    Scotland
Name: company_location, dtype: object>

In [61]:
teste=teste[teste['company_location']!= 'Scotland']

In [62]:
teste=teste.merge(continent[['country','continent','sub_region','code_2']].rename(columns={'continent':'bean_continent','sub_region':'bean_region','code_2':'bean_code_2'}), left_on='country_of_bean_origin', right_on='country', how='left')

In [63]:
teste[teste['bean_continent'].isna()]['country_of_bean_origin'].value_counts

<bound method IndexOpsMixin.value_counts of 88      Blend
92      Blend
96      Blend
100     Blend
177     Blend
        ...  
2188    Blend
2201    Blend
2203    Blend
2209    Blend
2213    Blend
Name: country_of_bean_origin, Length: 177, dtype: object>

In [64]:
teste=teste[teste['country_of_bean_origin']!= 'Blend']

In [65]:
data=teste

In [66]:
data.columns

Index(['ref', 'company', 'company_location', 'review_date',
       'country_of_bean_origin', 'specific_bean_origin_or_bar_name',
       'cocoa_percent', 'rating', 'counts_of_ingredients', 'beans',
       'cocoa_butter', 'vanilla', 'lecithin', 'salt', 'sugar',
       'sweetener_without_sugar', 'first_taste', 'second_taste', 'third_taste',
       'fourth_taste', 'country_x', 'company_continent', 'company_region',
       'company_code_2', 'country_y', 'bean_continent', 'bean_region',
       'bean_code_2'],
      dtype='object')

In [67]:
# Add the coordinates both for bean_origin and company_location
coord.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [68]:
teste1=data.merge(coord.iloc[:,:-1], left_on='bean_code_2', right_on='country', how='left').rename(columns={'latitude':'bean_lat','longitude':'bean_lon'})
teste2=data.merge(coord.iloc[:,:-1], left_on='company_code_2', right_on='country', how='left').rename(columns={'latitude':'country_lat','longitude':'country_lon'})

In [69]:
data1=data.groupby(by=['company']).mean()
data1

,ref,review_date,cocoa_percent,rating,counts_of_ingredients
company,,,,,
5150,2455.333333,2019.000000,76.000000,3.500000,3.000000
A. Morin,1294.480000,2013.960000,68.880000,3.400000,3.960000
AMMA,670.600000,2010.600000,69.000000,3.500000,4.000000
Acalli,1966.000000,2017.500000,71.000000,3.562500,2.000000
Adi aka Fijiana (Easy In Ltd),705.000000,2011.000000,75.000000,3.250000,3.000000
...,...,...,...,...,...
Zotter,1033.705882,2012.647059,70.352941,3.352941,3.823529
hello cocoa,1460.000000,2015.000000,65.500000,3.125000,3.000000
hexx,1848.000000,2016.500000,71.500000,2.925000,2.000000


In [70]:
var1=np.asarray(data1[data1.rating <= data1.rating.quantile(.25)]['cocoa_percent'],dtype=np.float32)
var2=np.asarray(data1[(data1.rating > data1.rating.quantile(.25)) & (data1.rating <= data1.rating.quantile(.50))]['cocoa_percent'],dtype=np.float32)
var3=np.asarray(data1[(data1.rating > data1.rating.quantile(.50)) & (data1.rating <= data1.rating.quantile(.75))]['cocoa_percent'],dtype=np.float32)
var4=np.asarray(data1[data1.rating > data1.rating.quantile(.75)]['cocoa_percent'],dtype=np.float32)

In [71]:
# falta meter titulo xaxis yaxis
#labels = ['1st quartile','2nd quartile','3rd quartile','4th quartile']
#variables=[var1,var2,var3,var4]
#fig = ff.create_distplot(variables,labels, show_hist=False)
#fig.show()


### Exports and Imports


In [72]:
imp_exp.head()

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD)
60,Albania,2012,"Cocoa beans, whole or broken, raw or roasted",Import,130.0,874.0
71,Albania,2011,"Cocoa beans, whole or broken, raw or roasted",Import,4524.0,19877.0
83,Albania,2010,"Cocoa beans, whole or broken, raw or roasted",Import,45.0,295.0
94,Albania,2009,"Cocoa beans, whole or broken, raw or roasted",Import,262.0,1919.0
149,Albania,2004,"Cocoa beans, whole or broken, raw or roasted",Import,1606.0,2729.0


In [73]:
imp_exp_regions = imp_exp.merge(continent[['country','continent','sub_region','code_2']].rename(columns={'sub_region':'region'}), left_on = 'Country or Area', right_on= 'country', how= 'left')

In [74]:
export = imp_exp_regions[imp_exp_regions['Flow'] == 'Export']
export.head()

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD),country,continent,region,code_2
8,Albania,1997,"Cocoa beans, whole or broken, raw or roasted",Export,3500.0,3447.00,Albania,Europe,Southern Europe,AL
51,Angola,2016,"Cocoa beans, whole or broken, raw or roasted",Export,10.0,25.75,Angola,Africa,Middle Africa,AO
79,Antigua and Barbuda,2000,"Cocoa beans, whole or broken, raw or roasted",Export,127.0,465.00,Antigua and Barbuda,Americas,Caribbean,AG
108,Argentina,1993,"Cocoa beans, whole or broken, raw or roasted",Export,132.0,420.00,Argentina,Americas,South America,AR
110,Armenia,2019,"Cocoa beans, whole or broken, raw or roasted",Export,23806.0,13462.21,Armenia,Asia,Western Asia,AM


In [75]:
export[export['continent'] == 'Europe']['Country or Area'].unique()

array(['Albania', 'Austria', 'Belarus', 'Belgium', 'Bulgaria', 'Croatia',
       'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
       'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Lithuania',
       'Luxembourg', 'Malta', 'Netherlands', 'Norway', 'Poland',
       'Portugal', 'Romania', 'Russian Federation', 'Serbia', 'Slovakia',
       'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Ukraine'],
      dtype=object)

In [76]:
imports = imp_exp_regions[imp_exp_regions['Flow'] == 'Import']
imports.head()

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD),country,continent,region,code_2
0,Albania,2012,"Cocoa beans, whole or broken, raw or roasted",Import,130.0,874.0,Albania,Europe,Southern Europe,AL
1,Albania,2011,"Cocoa beans, whole or broken, raw or roasted",Import,4524.0,19877.0,Albania,Europe,Southern Europe,AL
2,Albania,2010,"Cocoa beans, whole or broken, raw or roasted",Import,45.0,295.0,Albania,Europe,Southern Europe,AL
3,Albania,2009,"Cocoa beans, whole or broken, raw or roasted",Import,262.0,1919.0,Albania,Europe,Southern Europe,AL
4,Albania,2004,"Cocoa beans, whole or broken, raw or roasted",Import,1606.0,2729.0,Albania,Europe,Southern Europe,AL


In [77]:
pivot_imports = pd.pivot_table(imports, values = 'Trade (USD)', index = 'continent', columns = 'Year', aggfunc = np.sum)

In [78]:
pivot_imports.index.name = None
pivot_imports

Year,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Africa,NaN,NaN,NaN,572105.0,1944817.0,1259203.0,3.680804e+06,3.146172e+06,7.516860e+06,2.382868e+06,...,2.724588e+07,1.652407e+09,3.124253e+07,8.429142e+07,3.864340e+07,1.710996e+07,1.977022e+07,1.946710e+07,1.238301e+06,3.180144e+03
Americas,NaN,33847189.0,38439802.0,44099672.0,39999341.0,45966597.0,5.615292e+07,7.992683e+07,6.335198e+07,9.428702e+07,...,3.682923e+08,4.040635e+08,3.341691e+08,2.660147e+08,4.241278e+08,3.429008e+08,4.908743e+08,4.319500e+08,4.580726e+08,2.265479e+08
Asia,84981276.0,189457873.0,182257197.0,178398910.0,220703086.0,160839787.0,1.904380e+08,2.746586e+08,2.664382e+08,2.931037e+08,...,2.001048e+09,2.129123e+09,1.814508e+09,1.587756e+09,2.195441e+09,1.741891e+09,1.812279e+09,1.972022e+09,2.074764e+09,1.474099e+08
Europe,59869290.0,119171486.0,124005183.0,504019590.0,927868418.0,834262287.0,1.399687e+09,1.630405e+09,1.978299e+09,1.667038e+09,...,5.236534e+09,6.174367e+09,4.895103e+09,4.551503e+09,5.295596e+09,5.619806e+09,6.454687e+09,5.625416e+09,5.345948e+09,1.869602e+08
Oceania,981118.0,263575.0,179329.0,236528.0,346922.0,329263.0,3.435490e+05,4.244230e+05,3.892580e+05,4.929880e+05,...,6.344571e+06,6.481258e+06,6.484481e+06,5.499694e+06,6.399103e+06,1.216092e+07,9.947090e+06,8.159731e+06,8.891566e+06,NaN


In [79]:
pivot_export = pd.pivot_table(export, values = 'Trade (USD)', index = 'continent', columns = 'Year', aggfunc = np.sum)

In [80]:
pivot_export.index.name = None
pivot_export.index

Index(['Africa', 'Americas', 'Asia', 'Europe', 'Oceania'], dtype='object')

In [81]:
#frame1 = []
#frame2 = []

frames = []

for year in pivot_export.columns[1:]: # we do not include the first frame here
    frames.append(dict(data=dict(type='bar',
                                 x=pivot_export.index,
                                 y=pivot_export[year],
                                 name=year,
                                ),
                       layout=go.Layout(title_text='Exports in the year of ' + str(year))
                      )
                )

#for year in pivot_imports.columns[1:]: # we do not include the first frame here
#    frames.append(dict(data=dict(type='scatter',
#                                 x=pivot_imports.index,
#                                 y=pivot_imports[year],
#                                 name=year,
#                                ),
#                       layout=go.Layout(title_text='Imports in the year of ' + str(year))
#                      )
#                )

    
#frames = [frame1, frame2]

# First introduction to buttons
fig_bar_layout = dict(title=dict(text='Exports by continent from 1988 to 2019'),
                      yaxis=dict(title='Exports',
                                 range=[0,9000000000]
                                ),
                      updatemenus=[dict(type="buttons",
                                    buttons=[dict(label="Play",
                                                  method="animate",
                                                  args=[None])
                                            ]
                                       )
                                  ]
                    )

initial_data = dict(type='bar', 
                    x=pivot_export.index, 
                    y=pivot_export.values[0],
                    name=str(1988))

fig_bar = go.Figure(data = initial_data, layout = fig_bar_layout, frames=frames)

#fig_bar.show(renderer='iframe')

### ------------- BEA


In [82]:
imports.loc[(imports['continent']=='nan') & (imports['Country or Area']!='nan')]

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD),country,continent,region,code_2


In [83]:
imports['country'].isna().sum()

361

In [84]:
imports['continent'].isna().sum()

361

In [85]:
imports['Quantity'] = imports['Quantity']. fillna(0) #os que têm nans ficam considerados como 0 importação

In [86]:
imports.dtypes

Country or Area     object
Year                 int64
Commodity           object
Flow                object
Quantity           float64
Trade (USD)        float64
country             object
continent           object
region              object
code_2              object
dtype: object

In [87]:
imports['Quantity'].astype(int)

0         130
1        4524
2          45
3         262
4        1606
        ...  
5050       40
5051        8
5052      257
5054      221
5056    14942
Name: Quantity, Length: 2865, dtype: int32

In [88]:
#treemap: #px.Constant("continents")

In [89]:
imports=imports[imports['Country or Area'].isnull()!= True] #removing nulls from Country or Area Column

In [90]:
# Mudar aqui:

In [91]:
imports.loc[imports['Country or Area'] == 'Wallis and Futuna Isds', 'continent'] = 'Oceania'
imports.loc[imports['Country or Area'] == 'Wallis and Futuna Isds','region'] = 'Polynesia'
#imports.loc[imports['Country or Area'] == 'United Rep. of Tanzania','country'] = 'Tanzania'

In [92]:
imports.loc[imports['Country or Area'] == 'Wallis and Futuna Isds', 'continent'] = 'Oceania'
imports.loc[imports['Country or Area'] == 'Wallis and Futuna Isds','region'] = 'Polynesia'
imports.loc[imports['Country or Area'] == 'Venezuela', 'continent'] = 'Americas'
imports.loc[imports['Country or Area'] == 'Venezuela','region'] = 'Southern America'
imports.loc[imports['Country or Area'] == 'United Rep. of Tanzania', 'continent'] = 'Africa'
imports.loc[imports['Country or Area'] == 'United Rep. of Tanzania','region'] = 'Eastern Africa'
imports.loc[imports['Country or Area'] == 'United Rep. of Tanzania','country'] = 'Tanzania'
imports.loc[imports['Country or Area'] == 'United Kingdom', 'continent'] = 'Europe'
imports.loc[imports['Country or Area'] == 'United Kingdom','region'] = 'Northern Europe'
imports.loc[imports['Country or Area'] == 'USA', 'continent'] = 'Americas'
imports.loc[imports['Country or Area'] == 'USA','region'] = 'Northern America'
imports.loc[imports['Country or Area'] == 'USA','country'] = 'United States of America'
imports.loc[imports['Country or Area'] == 'Turks and Caicos Isds', 'continent'] = 'Americas'
imports.loc[imports['Country or Area'] == 'Turks and Caicos Isds','region'] = 'Central America'
imports.loc[imports['Country or Area'] == 'TFYR of Macedonia', 'continent'] = 'Europe'
imports.loc[imports['Country or Area'] == 'TFYR of Macedonia','region'] = 'Southern Europe'
imports.loc[imports['Country or Area'] == 'TFYR of Macedonia','country'] = 'Macedonia'
imports.loc[imports['Country or Area'] == 'Syria', 'continent'] = 'Asia'
imports.loc[imports['Country or Area'] == 'Syria','region'] = 'Western Asia'
imports.loc[imports['Country or Area'] == 'Solomon Isds', 'continent'] = 'Oceania'
imports.loc[imports['Country or Area'] == 'Solomon Isds','region'] = 'Australia and New Zealand'
imports.loc[imports['Country or Area'] == 'Serbia and Montenegro', 'continent'] = 'Europe'
imports.loc[imports['Country or Area'] == 'Serbia and Montenegro','region'] = 'Southern Europe'
imports.loc[imports['Country or Area'] == 'Rep. of Moldova', 'continent'] = 'Europe'
imports.loc[imports['Country or Area'] == 'Rep. of Moldova','region'] = 'Eastern Europe'
imports.loc[imports['Country or Area'] == 'Rep. of Moldova','country'] = 'Moldova'
imports.loc[imports['Country or Area'] == 'Rep. of Korea', 'continent'] = 'Asia'
imports.loc[imports['Country or Area'] == 'Rep. of Korea','region'] = 'Eastern Asia'
imports.loc[imports['Country or Area'] == 'Rep. of Korea','country'] = 'South Korea'
imports.loc[imports['Country or Area'] == 'Other Asia, nes', 'continent'] = 'Asia'
imports.loc[imports['Country or Area'] == 'Other Asia, nes','region'] = 'Other'
imports.loc[imports['Country or Area'] == 'Neth. Antilles', 'continent'] = 'Americas'
imports.loc[imports['Country or Area'] == 'Neth. Antilles','region'] = 'Caribbean'
imports.loc[imports['Country or Area'] == 'Lao People\'s Dem. Rep.', 'continent'] = 'Asia'
imports.loc[imports['Country or Area'] == 'Lao People\'s Dem. Rep.','region'] = 'South-Eastern Asia'
imports.loc[imports['Country or Area'] == 'Lao People\'s Dem. Rep.','country'] = 'Laos'
imports.loc[imports['Country or Area'] == 'Iran', 'continent'] = 'Asia'
imports.loc[imports['Country or Area'] == 'Iran','region'] = 'South-Western Asia'
imports.loc[imports['Country or Area'] == 'Fmr Fed. Rep. of Germany', 'continent'] = 'Europe'
imports.loc[imports['Country or Area'] == 'Fmr Fed. Rep. of Germany','region'] = 'Central Europe'
imports.loc[imports['Country or Area'] == 'Fmr Fed. Rep. of Germany','country'] = 'Germany'
imports.loc[imports['Country or Area'] == 'Faeroe Isds', 'continent'] = 'Europe'
imports.loc[imports['Country or Area'] == 'Faeroe Isds', 'region'] = 'Northern Europe'
imports.loc[imports['Country or Area'] == 'FS Micronesia', 'continent'] = 'Oceania'
imports.loc[imports['Country or Area'] == 'FS Micronesia', 'region'] = 'Micronesia'
imports.loc[imports['Country or Area'] == 'Czech Rep.', 'continent'] = 'Europe'
imports.loc[imports['Country or Area'] == 'Czech Rep.', 'region'] = 'Central Europe'
imports.loc[imports['Country or Area'] == 'Bosnia Herzegovina', 'continent'] = 'Europe'
imports.loc[imports['Country or Area'] == 'Bosnia Herzegovina', 'region'] = 'Southern Europe'
imports.loc[imports['Country or Area'] == 'Central African Rep.', 'continent'] = 'Africa'
imports.loc[imports['Country or Area'] == 'Central African Rep.', 'region'] = 'Middle Africa'
imports.loc[imports['Country or Area'] == 'China, Hong Kong SAR', 'continent'] = 'Asia'
imports.loc[imports['Country or Area'] == 'China, Hong Kong SAR', 'region'] = 'Eastern Asia'
imports.loc[imports['Country or Area'] == 'China, Macao SAR', 'continent'] = 'Asia'
imports.loc[imports['Country or Area'] == 'China, Macao SAR', 'region'] = 'Eastern Asia'
imports.loc[imports['Country or Area'] == 'Cook Isds', 'continent'] = 'Oceania'
imports.loc[imports['Country or Area'] == 'Cook Isds', 'region'] = 'Polynesia'
imports.loc[imports['Country or Area'] == 'Dominican Rep.', 'continent'] = 'Americas'
imports.loc[imports['Country or Area'] == 'Dominican Rep.', 'region'] = 'Caribbean'
imports.loc[imports['Country or Area'] == 'Fmr Sudan', 'continent'] = 'Africa'
imports.loc[imports['Country or Area'] == 'Fmr Sudan','region'] = 'Northern Africa'
imports.loc[imports['Country or Area'] == 'Fmr Sudan','country'] = 'Sudan'

In [93]:
imports= imports[~(imports['Country or Area'] == 'Belgium-Luxembourg')]

In [94]:
imports= imports[~(imports['Country or Area'] == 'EU-28')]
imports= imports[~(imports['Country or Area'] == 'So. African Customs Union')]

In [95]:
fig = px.treemap(imports, 
    path=['continent', 'region', 'Country or Area'],values='Quantity', color='Quantity', color_continuous_scale='sunset'

)

fig.update_traces(root_color="lightgrey")

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25)) #,paper_bgcolor='rgb(233,233,233)')
fig.show()


C:\Users\Utilizador\anaconda3\envs\datavisualization\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Utilizador\anaconda3\envs\datavisualization\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Utilizador\anaconda3\envs\datavisualization\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



# ----- Helena


In [96]:
yes_ingredients = [{'label': 'Vanilla', 'value': 'vanilla'},
           {'label': 'Lecithin', 'value': 'lecithin'},
           {'label': 'Salt', 'value': 'salt'},
           {'label': 'Sweetener without sugar', 'value': 'sweetener_without_sugar'},
           {'label': 'Sugar', 'value': 'sugar'}]

In [97]:
radio_sugar = [
    {'label': 'Has Sugar', 'value': 'have_sugar'},
    {'label': 'Has not Sugar', 'value': 'have_not_sugar'}
]

radio_vanilla = [
    {'label': 'Has Vanilla', 'value': 'have_vanilla'},
    {'label': 'Has not Vanilla', 'value': 'have_not_vanilla'}
]

radio_salt = [
    {'label': 'Has Salt', 'value': 'have_salt'},
    {'label': 'Has not Salt', 'value': 'have_not_salt'}
]

radio_lecithin = [
    {'label': 'Has Lecithin', 'value': 'have_lecithin'},
    {'label': 'Has not Lecithin', 'value': 'have_not_lecithin'}
]


In [48]:
ing_checkbox = dcc.Checklist(
        id='ing_checkbox',
        options=yes_ingredients,
        value=['sugar']
    ),

In [98]:

ingredients = [
    {'label': 'Has Sugar', 'value': 'have_sugar'},
    {'label': 'Has not Sugar', 'value': 'have_not_sugar'},
    {'label': 'Has Vanilla', 'value': 'have_vanila'},
    {'label': 'Has not Vanilla', 'value': 'have_not_vanila'},
    {'label': 'Has Salt', 'value': 'have_salt'},
    {'label': 'Has not Salt', 'value': 'have_not_salt'},
    {'label': 'Has Lecithin', 'value': 'have_lecithin'},
    {'label': 'Has not Lecithin', 'value': 'have_not_lecithin'}
]


In [99]:

# The app itself

app = dash.Dash(__name__)

app.layout = html.Div([

    html.H4('Choose your chocolate'),
        
        html.Div([
                      
            html.Div([
                html.Label('The ingredients'),
                    dcc.Dropdown(
                                    id='drop_id',
                                    options=ingredients,
                                    value=['have_sugar'],
                                    multi=True
                                ),
            ], className='box', style={'margin': '10px', 'padding-top':'15px', 'padding-bottom':'15px'}),
            
            html.Div([
                html.Label('Cocoa Percentage'),
                    dcc.RangeSlider(
                                    id='percent_id',
                                    min=42,
                                    max=100,
                                    value=[70, 80],
                                    marks={'50': '50',
                                        '60': '60',
                                        '70': '70',
                                        '80': '80',
                                        '90': '90',
                                        '100': '100'},
                                    step=1
                                )]),
            
            
            html.Div([
                html.Div([
                    html.Label('Results', style={'font-size': 'medium'}),
                    html.Br(),
                    html.Br(),
                    html.Div([
                        
                        html.Div([
                            html.H4('Company', style={'font-weight':'normal'}),
                            html.H3(id="name_company_id")
                        ],className='box_ing'),
                        
                        html.Div([
                            html.H4('Rating', style={'font-weight':'normal'}),
                            html.H3(id="rating_id")]),
                        ],className='box_ing'),

                        html.Div([
                            html.H4('Country', style={'font-weight':'normal'}),
                             html.H3(id="country_id"),
                        ],className='box_ing'),
                    
                        ], style={'display': 'flex'}),

                    ], className='box', style={'heigth':'10%'}),
 
                html.Div([ 
                    html.Div([
                        
                        html.Div([
                            html.Br(),
                            html.Label(id='title_map', style={'font-size':'medium'}), 
                            html.Br(),
                            html.Label('BLA BLA BLA', style={'font-size':'9px'}),
                        ], style={'width': '70%'}),
                        html.Div([

                        ], style={'width': '5%'}),
                       
                    
                    dcc.Graph(id='graph1'),

                ], className='box', style={'padding-bottom': '0px'}), 
                    ]),
                ], style={'width': '60%'}),           
            ], className='row')
    

In [100]:
@app.callback(
   
   [Output("name_company_id", "children"),
    Output("rating_id", "children"),
    Output("country_id", "children"),
    Output('graph1', 'figure')],
   
    [Input('drop_id', 'value'),
     Input('percent_id','value')] )

def update_graph(drop_id,percent_id):
    
    filtered=data
    for each in drop_id:
        filtered=filtered[filtered.isin([each]).any(1)] 
    
    filtered = filtered[(filtered['cocoa_percent'] >= percent_id[0]) & (filtered['cocoa_percent'] <= percent_id[1])]

    # maxi=np.max(filtered['rating'])
    # filtered=filtered[filtered['rating']==maxi]
    
    data_inter = dict(type='bar', x=filtered['company'], y=filtered['rating'])
   
    layout_1 = dict(xaxis=dict(title='Companies'),
                   yaxis=dict(title='Ratings'))
    
    fig = go.Figure(data=data_inter, layout=layout_1)  

    if filtered.empty:
        name_company = 'No company was found'
        rating = 'No rating was found'
        country= 'No country was found'
        
    else:
        name_company=str(filtered.head(1)['company'].values[0])
        rating=str(filtered.head(1)['rating'].values[0])
        country=str(filtered.head(1)['company_location'].values[0])
    
    return name_company ,  \
           rating , \
           country , \
           fig

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


# ------- Filipa


### Radar Plot


In [101]:
radar = pd.DataFrame(round(data.groupby(by = 'company')[['cocoa_percent', 'rating', 'counts_of_ingredients']].mean(),2))
radar['company_name'] = radar.index
radar['cocoa_percent'] = radar['cocoa_percent'] / 100
radar

,cocoa_percent,rating,counts_of_ingredients,company_name
company,,,,
5150,0.7600,3.50,3.00,5150
A. Morin,0.6888,3.40,3.96,A. Morin
AMMA,0.6900,3.50,4.00,AMMA
Acalli,0.7100,3.56,2.00,Acalli
Adi aka Fijiana (Easy In Ltd),0.7500,3.25,3.00,Adi aka Fijiana (Easy In Ltd)
...,...,...,...,...
Zotter,0.7035,3.35,3.82,Zotter
hello cocoa,0.6550,3.12,3.00,hello cocoa
hexx,0.7150,2.92,2.00,hexx


In [159]:
feat_radar = ['cocoa_percent', 'rating', 'counts_of_ingredients']

In [103]:
def radar_plot(company1, company2):

    # scatterpolar
    company1_df = pd.DataFrame(radar[radar['company_name'] == company1][feat_radar].iloc[0])
    company1_df.columns = ['total']
    company2_df = pd.DataFrame(radar[radar['company_name'] == company2][feat_radar].iloc[0])
    company2_df.columns = ['total']
    total_list = [company1_df.index[i].capitalize() + ' = ' + str(company1_df['total'][i]) for i in range(len(company1_df))]
    total1_text = company1
    for i in total_list:
        total1_text += '<br>' + i

    total_list = [company2_df.index[i].capitalize() + ' = ' + str(company2_df['total'][i]) for i in range(len(company2_df))]
    total2_text = company2
    for i in total_list:
        total2_text += '<br>' + i

    fig = go.Figure(data=go.Scatterpolar(
        r=company1_df['total'],
        theta=company1_df.index,
        fill='toself', 
        marker_color = 'rgb(45,0,198)',   
        opacity =1, 
        hoverinfo = "text" ,
        name = total1_text,
        text  = [company1_df.index[i] +' = ' + str(company1_df['total'][i]) for i in range(len(company1_df))]
    ))
    fig.add_trace(go.Scatterpolar(
        r=company2_df['total'],
        theta=company2_df.index,
        fill='toself',
        marker_color = 'rgb(255,171,0)',
        hoverinfo = "text" ,
        name= total2_text,
        text  = [company2_df.index[i] +' = ' + str(company2_df['total'][i]) for i in range(len(company2_df))]
        ))

    fig.update_layout(
        polar=dict(
            hole=0.1,
            bgcolor="white",
            radialaxis=dict(
                visible=True,
                type='linear',
                autotypenumbers='strict',
                autorange=False,
                range=[30, 100],
                angle=90,
                showline=False,
                showticklabels=False, ticks='',
                gridcolor='black'),
                ),
        width = 550,
        height = 550,
        margin=dict(l=80, r=80, t=20, b=20),
        showlegend=False,
        template="plotly_dark",
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        paper_bgcolor = 'rgba(0, 0, 0, 0)',
        font_color="black",
        font_size= 15
    )
    
    return fig


In [116]:
company1 = '5150'
company2 = 'A. Morin'
radar_plot(company1, company2)

KeyError: "['count_tastes'] not in index"

In [117]:
#company1_list = radar[radar['company_name'] == company1][feat_radar].values
company1_list = []

company1_df = pd.DataFrame(radar[radar['company_name'] == company1])
for i in range(len(radar.columns)-1):
    company1_list.append(radar[radar['company_name'] == company1].iloc[0,i])

company1_list

[0.76, 3.5, 3.0]

In [118]:
company2_list = []

company2_df = pd.DataFrame(radar[radar['company_name'] == company2])
for i in range(len(radar.columns)-1):
    company2_list.append(radar[radar['company_name'] == company2].iloc[0,i])

company2_list

[0.6888, 3.4, 3.96]

In [60]:

fig = go.Figure(data=go.Scatterpolar(
        r=company1_list,
        theta=feat_radar,
        fill='toself', 
        marker_color = 'rgb(45,0,198)',   
        opacity =1, 
        hoverinfo = "text" ,
        name = company1
        #text  = [company1_df.index[i] +' = ' + str(company1_df['total'][i]) for i in range(len(company1_df))]
    ))
fig.add_trace(go.Scatterpolar(
        r=company2_list,
        theta=feat_radar,
        fill='toself',
        marker_color = 'rgb(255,171,0)',
        hoverinfo = "text" ,
        name= company2,
        #text  = [radar.columns()[i] +' = ' + str(radar['total'][i]) for i in range(len(company2_df))]
        ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

fig.show()

### Column count_tastes


In [160]:
test_taste = data
test_taste.head()

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,beans,...,company_code_2,country_y,bean_continent,bean_region,bean_code_2,binFirst_taste,binSecond_taste,binThird_taste,binFourth_taste,count_tastes
0,2454,5150,United States of America,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.75,3,have_bean,...,US,Madagascar,Africa,Eastern Africa,MG,1,1,1,0,3
1,2458,5150,United States of America,2019,Dominican Republic,"Zorzal, batch 1",76.0,3.50,3,have_bean,...,US,Dominican Republic,Americas,Caribbean,DO,1,1,1,0,3
2,2454,5150,United States of America,2019,"Tanzania, United Republic of","Kokoa Kamili, batch 1",76.0,3.25,3,have_bean,...,US,"Tanzania, United Republic of",Africa,Eastern Africa,TZ,1,1,1,0,3
3,797,A. Morin,France,2012,Peru,Peru,63.0,3.75,4,have_bean,...,FR,Peru,Americas,South America,PE,1,1,1,0,3
4,797,A. Morin,France,2012,Bolivia (Plurinational State of),Bolivia,70.0,3.50,4,have_bean,...,FR,Bolivia (Plurinational State of),Americas,South America,BO,1,1,0,0,2


In [161]:
test_taste['first_taste'].fillna(value = 0, inplace = True)
test_taste['second_taste'].fillna(value = 0, inplace = True)
test_taste['third_taste'].fillna(value = 0, inplace = True)
test_taste['fourth_taste'].fillna(value = 0, inplace = True)

In [162]:
#taste = lambda x: 1 if x.isna() == False else 0
taste = lambda x: 1 if x != 0 else x

In [163]:
test_taste['binFirst_taste'] = test_taste['first_taste'].apply(taste)
test_taste['binSecond_taste'] = test_taste['second_taste'].apply(taste)
test_taste['binThird_taste'] = test_taste['third_taste'].apply(taste)
test_taste['binFourth_taste'] = test_taste['fourth_taste'].apply(taste)

In [164]:
test_taste[test_taste['binThird_taste'] == 0].head()

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,beans,...,company_code_2,country_y,bean_continent,bean_region,bean_code_2,binFirst_taste,binSecond_taste,binThird_taste,binFourth_taste,count_tastes
4,797,A. Morin,France,2012,Bolivia (Plurinational State of),Bolivia,70.0,3.50,4,have_bean,...,FR,Bolivia (Plurinational State of),Americas,South America,BO,1,1,0,0,2
9,1011,A. Morin,France,2013,Brazil,Brazil,70.0,3.25,4,have_bean,...,FR,Brazil,Americas,South America,BR,1,0,0,0,1
10,1015,A. Morin,France,2013,Papua New Guinea,Papua New Guinea,70.0,3.25,4,have_bean,...,FR,Papua New Guinea,Oceania,Melanesia,PG,1,1,0,0,2
15,1015,A. Morin,France,2013,Colombia,Colombie,70.0,2.75,4,have_bean,...,FR,Colombia,Americas,South America,CO,1,1,0,0,2
18,1315,A. Morin,France,2014,Cuba,Cuba,70.0,3.50,4,have_bean,...,FR,Cuba,Americas,Caribbean,CU,1,1,0,0,2


In [165]:
test_taste['count_tastes'] = test_taste['binFirst_taste'] + test_taste['binSecond_taste'] + test_taste['binThird_taste'] + test_taste['binFourth_taste']

In [166]:
test_taste.head()

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,beans,...,company_code_2,country_y,bean_continent,bean_region,bean_code_2,binFirst_taste,binSecond_taste,binThird_taste,binFourth_taste,count_tastes
0,2454,5150,United States of America,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.75,3,have_bean,...,US,Madagascar,Africa,Eastern Africa,MG,1,1,1,0,3
1,2458,5150,United States of America,2019,Dominican Republic,"Zorzal, batch 1",76.0,3.50,3,have_bean,...,US,Dominican Republic,Americas,Caribbean,DO,1,1,1,0,3
2,2454,5150,United States of America,2019,"Tanzania, United Republic of","Kokoa Kamili, batch 1",76.0,3.25,3,have_bean,...,US,"Tanzania, United Republic of",Africa,Eastern Africa,TZ,1,1,1,0,3
3,797,A. Morin,France,2012,Peru,Peru,63.0,3.75,4,have_bean,...,FR,Peru,Americas,South America,PE,1,1,1,0,3
4,797,A. Morin,France,2012,Bolivia (Plurinational State of),Bolivia,70.0,3.50,4,have_bean,...,FR,Bolivia (Plurinational State of),Americas,South America,BO,1,1,0,0,2


### Parallel Coordinates Plot


In [167]:
company1 = 'Ah Cacao'
company2 = 'Zotter'


In [152]:
data.company

0           5150
1           5150
2           5150
3       A. Morin
4       A. Morin
          ...   
2207      Zotter
2208      Zotter
2210      Zotter
2211      Zotter
2212      Zotter
Name: company, Length: 2074, dtype: object

In [153]:
# feat_radar.append('count_tastes')
# feat_radar

['cocoa_percent', 'rating', 'counts_of_ingredients', 'count_tastes']

In [168]:
parallel_coord = test_taste[test_taste['company'].isin([company1,company2])]

In [169]:
filter_company = lambda x: 1 if x == company1 else 2

In [170]:
parallel_coord['company_number'] = parallel_coord['company'].apply(filter_company)

In [171]:
parallel_coord.head()

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,beans,...,country_y,bean_continent,bean_region,bean_code_2,binFirst_taste,binSecond_taste,binThird_taste,binFourth_taste,count_tastes,company_number
42,316,Ah Cacao,Mexico,2009,Mexico,Tabasco,70.0,3.00,3,have_bean,...,Mexico,Americas,Central America,MX,1,1,1,0,3,1
2193,486,Zotter,Austria,2010,Brazil,"Brazil, Mitzi Blue",65.0,3.00,4,have_bean,...,Brazil,Americas,South America,BR,1,1,0,0,2,2
2194,647,Zotter,Austria,2011,Peru,Peru,70.0,3.75,4,have_bean,...,Peru,Americas,South America,PE,1,1,1,0,3,2
2195,749,Zotter,Austria,2011,India,Kerala State,65.0,3.50,4,have_bean,...,India,Asia,Southern Asia,IN,1,1,1,0,3,2
2196,781,Zotter,Austria,2011,India,Kerala State,62.0,3.25,4,have_bean,...,India,Asia,Southern Asia,IN,1,1,1,1,4,2


In [172]:
fig = px.parallel_coordinates(parallel_coord,
                              dimensions = feat_radar,
                              color= parallel_coord['company_number'],
                              range_color=[0.5, 3.5],
                              color_continuous_scale=[(0.00, '#AB63FA'),   (0.5,'#AB63FA'),
                                                     (0.5, '#FFA15A'), (1, '#FFA15A')],
                              labels={"cocoa_percent": "Percentage of Cocoa (%)", 
                                      "rating": "Rating",
                                      "counts_of_ingredients": "Number of Ingredients", 
                                      "count_tastes": "Number of Tastes"},
                             color_continuous_midpoint=2,
                             title = f"Comparison between {company1} and {company2}"
                             )
fig.update_layout(
    coloraxis_colorbar=dict(
    title="Company",
    tickvals=[1,2],
    ticktext=[company1,company2],
    lenmode="pixels", len=100
))
fig.show()
